In [ ]:
!pip install tensorflow

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi


In [ ]:
img_path='/content/drive/MyDrive/red spider final'


In [ ]:
print(img_path)
import os
labels=[]
for i in os.listdir(img_path):
  if os.path.isdir(os.path.join(img_path,i)):
    labels.append(i)
print(labels)

In [ ]:
'''from tensorflow.keras.utils import image_dataset_from_directory # Import the required function

# Load dataset and split into train, validation, and test sets
train_ds = image_dataset_from_directory(
    img_path,
    validation_split=0.3,  # 20% of data will be used for validation/testing
    subset="training",
    seed=123,
    image_size=(224, 224),  # Resizing images for MobileNetV2

)'''

In [ ]:
'''val_test_ds = tf.keras.utils.image_dataset_from_directory(
    img_path,
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=BATCHSIZE
)'''

In [ ]:
'''
val_size = int(0.5 * tf.data.experimental.cardinality(val_test_ds).numpy())  # ~132 images
val_ds = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)

# Verify sizes (approximate due to batching)
print(f"Train size: {tf.data.experimental.cardinality(train_ds).numpy() * BATCHSIZE}")
print(f"Val size: {tf.data.experimental.cardinality(val_ds).numpy() * BATCHSIZE}")
print(f"Test size: {tf.data.experimental.cardinality(test_ds).numpy() * BATCHSIZE}")
'''

In [ ]:
BATCHSIZE=32
IMG_SIZE=224
EPOCHS=100

In [ ]:
data_dir = "/content/drive/MyDrive/red spider final"

In [ ]:
import tensorflow as tf

# Load dataset
full_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    seed=123,
    image_size=(IMG_SIZE,IMG_SIZE),
  #  seed=123,
    batch_size=BATCHSIZE

)

# Unbatch and get exact count
images_labels = list(full_ds.unbatch())
total_images = len(images_labels)  # 879
print(f"Total images: {total_images}")

# Separate images and labels
images = [x[0] for x in images_labels]
labels = [x[1] for x in images_labels]

# Exact split
train_size = int(0.7 * total_images)  # 615
val_size = int(0.15 * total_images)   # 131
test_size = total_images - train_size - val_size  # 133

# Create datasets without padding
train_ds = tf.data.Dataset.from_tensor_slices(
    (images[:train_size], labels[:train_size])
).batch(BATCHSIZE, drop_remainder=True)  # Drop partial batches

val_ds = tf.data.Dataset.from_tensor_slices(
    (images[train_size:train_size + val_size], labels[train_size:train_size + val_size])
).batch(BATCHSIZE, drop_remainder=True)

test_ds = tf.data.Dataset.from_tensor_slices(
    (images[train_size + val_size:], labels[train_size + val_size:])
).batch(BATCHSIZE, drop_remainder=True)



In [ ]:
# Print sizes
print(f"Train size: {tf.data.experimental.cardinality(train_ds).numpy() * BATCHSIZE}")
print(f"Val size: {tf.data.experimental.cardinality(val_ds).numpy() *BATCHSIZE}")
print(f"Test size: {tf.data.experimental.cardinality(test_ds).numpy() * BATCHSIZE}")

# Verify actual counts
print(f"Actual Train: {len(list(train_ds.unbatch()))}")
print(f"Actual Val: {len(list(val_ds.unbatch()))}")
print(f"Actual Test: {len(list(test_ds.unbatch()))}")

In [ ]:
class_names = full_ds.class_names
print(class_names)

In [ ]:
#visualization
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(5):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")


In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
normalizations_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds=train_ds.map(lambda x,y:(normalizations_layer(x),y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:

data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal', input_shape=(IMG_SIZE, IMG_SIZE, 3)),  # Random horizontal flip
    layers.RandomRotation(0.1),  # Random rotation (-20% to +20%)
   # layers.RandomBrightness(factor=(0.7, 1.3)),  # Adjust brightness in the range 0.7–1.3
    layers.RandomZoom(0.1)  # Random zoom by 5%
])

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),

                                            include_top=False, weights='imagenet')

base_model.trainable = True
for layer in base_model.layers[:120]:
    layer.trainable = False


In [ ]:
#base_model.summary()

In [ ]:
#setupe the model architecture
l2_lambda =0.005
inputs=keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))
X=data_augmentation(inputs)
x=keras.applications.mobilenet_v2.preprocess_input(X)
x=base_model(x)
x = layers.GlobalAveragePooling2D()(x)
x=layers.Dropout(0.5)(x)
#x=layers.Dense(512)(x)
#x=layers.BatchNormalization()(x)
#x = layers.ReLU()(x)

x=layers.Dense(256,kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))(x)

x=layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x=layers.Dense(128,kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))(x)
x=layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x=layers.Dropout(0.6)(x)   #0.5


#output layar
# The prediction layer should match the output of the preceding layer.
prediction_layer = tf.keras.layers.Dense(4, activation="softmax")
outputs=prediction_layer(x)
model=keras.Model(inputs,outputs)

In [ ]:
#model.summary()

In [ ]:
callbacks=[
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3

    )

]

In [ ]:
import numpy as np

# Convert labels to NumPy array before concatenation
labels = np.array([y.numpy() for _, y in train_ds.unbatch()])

# Now you can use bincount
print(np.bincount(labels))  # بيطبع عدد الصور لكل فئة


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


# Ensure 'classes' is a NumPy array
class_weights = compute_class_weight('balanced', classes=np.array([0, 1, 2, 3]), y=labels)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
lr=1e-4

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights_dict

)

In [ ]:
score = model.evaluate(test_ds)

In [ ]:
model.save('red_spider_91.35.h5')

In [ ]:
#convert the model into tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('red_spider_91.35.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs_range=range(EPOCHS)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(5):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        plt.axis("off")

In [ ]:


import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Assuming 'model' and 'test_ds' are defined from the previous code

# Get predictions on the test set
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Add labels to each cell
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


In [ ]:
# print Precision, Recall, F1-score
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred_classes))


In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False